In [15]:
%%capture
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

In [16]:
import logging

import holoviews as hv
import hvplot.polars  # noqa
import polars as pl
from icecream import ic
from polars import col

from src.database.database_manager import DatabaseManager
from src.database.quality_checks import check_sample_rate
from src.features.resampling import decimate, interpolate_and_fill_nulls
from src.features.scaling import scale_min_max
from src.features.transforming import map_trials, merge_dfs
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
)

configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [17]:
db = DatabaseManager()

In [18]:
with db:
    face = db.get_table("Preprocess_Face", exclude_invalid_data=False)
    stimulus = db.get_table("Preprocess_Stimulus", exclude_invalid_data=False)
df = merge_dfs([face, stimulus])
df = interpolate_and_fill_nulls(df)
df

trial_id,trial_number,participant_id,rownumber,timestamp,samplenumber,anger,contempt,disgust,fear,joy,sadness,surprise,engagement,valence,sentimentality,confusion,neutral,attention,brow_furrow,brow_raise,cheek_raise,chin_raise,dimpler,eye_closure,eye_widen,inner_brow_raise,jaw_drop,lip_corner_depressor,lip_press,lip_pucker,lip_stretch,lip_suck,lid_tighten,mouth_open,nose_wrinkle,smile,smirk,upper_lip_raise,blink,blinkrate,pitch,yaw,roll,interocular_distance,temperature,rating
u16,u8,u8,u32,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64
1,1,1,null,294224.331,null,0.00125,0.00182,0.00024,0.00024,0.00026,0.00125,0.00061,0.00329,0.0,0.00036,0.00036,0.99636,0.98485,0.00036,0.00436,0.00286,0.0,0.00341,0.0,0.00066,0.00086,0.01865,0.0,0.00002,0.0,0.00006,0.00001,0.0,0.0048,0.00007,0.00642,0.0,0.0,null,null,-0.2371,-0.01128,0.03344,99.168541,0.0,0.425
1,1,1,8811,294350.9838,8803,0.00125,0.00182,0.00024,0.00024,0.00026,0.00125,0.00061,0.00329,0.0,0.00036,0.00036,0.99636,0.98485,0.00036,0.00436,0.00286,0.0,0.00341,0.0,0.00066,0.00086,0.01865,0.0,0.00002,0.0,0.00006,0.00001,0.0,0.0048,0.00007,0.00642,0.0,0.0,0,18,-0.2371,-0.01128,0.03344,99.168541,0.000066,0.425
1,1,1,null,294357.9645,null,0.001251,0.001821,0.00024,0.00024,0.00026,0.00125,0.00061,0.00329,0.0,0.000357,0.000357,0.996356,0.984949,0.000357,0.00432,0.002782,0.0,0.003205,0.0,0.000685,0.000835,0.018604,0.0,0.000019,0.0,0.000058,0.00001,0.0,0.004713,0.000069,0.006298,0.0,0.0,null,null,-0.237843,-0.010933,0.032799,99.128076,0.000069,0.425
1,1,1,8814,294450.0503,8806,0.00126,0.00184,0.00024,0.00024,0.00026,0.00125,0.00061,0.00329,0.0,0.00032,0.00032,0.99631,0.98625,0.00032,0.00379,0.00176,0.0,0.0005,0.0,0.00102,0.00051,0.018,0.0,0.00001,0.0,0.00003,0.00001,0.0,0.00356,0.00006,0.00469,0.0,0.0,0,18,-0.24765,-0.00636,0.02435,98.594276,0.00026,0.359431
1,1,1,null,294458.0292,null,0.001261,0.001843,0.00024,0.00024,0.000259,0.001251,0.000608,0.00329,0.0,0.000351,0.000351,0.996304,0.986232,0.000351,0.003776,0.001672,0.0,0.00046,8.0124e-7,0.001039,0.000508,0.01712,0.0,0.000009,0.0,0.000028,0.000009,0.0,0.003388,0.000061,0.004479,0.0,0.0,null,null,-0.246434,-0.005274,0.024485,98.511204,0.000277,0.35375
1,1,1,8817,294549.6316,8809,0.00127,0.00188,0.00024,0.00024,0.00025,0.00126,0.00058,0.00329,0.0,0.00071,0.00071,0.99623,0.98602,0.00071,0.00361,0.00066,0.0,0.0,0.00001,0.00126,0.00048,0.00702,0.0,0.0,0.0,0.00001,0.0,0.0,0.00141,0.00007,0.00206,0.0,0.0,0,18,-0.23247,0.00719,0.02604,97.55748,0.000591,0.167032
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
332,12,28,83114,2.7769e6,83104,0.00533,0.00476,0.00028,0.00069,0.00024,0.00644,0.00061,0.01464,-0.06243,0.0,0.02195,0.98712,0.98451,0.36278,0.0,0.00001,0.0,0.0,0.0,0.00002,0.00001,0.02195,0.0,0.0,0.00001,0.0,0.0,0.00071,0.04675,0.00262,0.0,0.00001,0.00007,0,36,-0.14984,-0.01242,-0.01782,106.573822,0.155973,0.85
332,12,28,null,2.7769e6,null,0.004642,0.004378,0.000274,0.000625,0.00024,0.005564,0.000622,0.011149,-0.042445,0.0,0.028302,0.988865,0.984945,0.328157,0.0,0.00001,0.0,0.0,0.0,0.000014,0.000004,0.028302,0.0,0.0,0.00001,0.0,0.0,0.000592,0.038729,0.002344,0.0,0.000016,0.000058,null,null,-0.138293,-0.010957,-0.018466,106.087479,0.155637,0.85


### "lowering of the eyebrows, squeezing of the eyes, wrinkling of the nose, raising of the upper lip and
opening of the mouth" (copied from Kappesser, 2019)

In [19]:
features = [
    "brow_furrow",
    "cheek_raise",
    "mouth_open",
    "upper_lip_raise",
    "nose_wrinkle",
]


In [ ]:
features = [
    "brow_furrow",
    "cheek_raise",
    "mouth_open",
    "upper_lip_raise",
    "nose_wrinkle",
]

for feature in features:
    col1, col2 = feature, "rating"

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
    )
    display(plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr"))
    display(
        plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")
    )


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [21]:
df.hvplot(x="timestamp", y=[col1, col2], groupby="trial_id", ylim=(0, 1))

BokehModel(combine_events=True, render_bundle={'docs_json': {'b0df907b-db31-40f7-bc96-f3b54e3d1174': {'version…

In [ ]:
further_features = [
    "anger",
    "contempt",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",
    "engagement",
    "valence",
    "sentimentality",
    "confusion",
    "neutral",
    "attention",
    "brow_raise",
    "chin_raise",
    "dimpler",
    "eye_closure",
    "eye_widen",
    "inner_brow_raise",
    "jaw_drop",
    "lip_corner_depressor",
    "lip_press",
    "lip_pucker",
    "lip_stretch",
    "lip_suck",
    "lid_tighten",
    "smile",
    "smirk",
    "blink",
    "blinkrate",
    "pitch",
    "yaw",
    "roll",
    "interocular_distance",
]

for feature in further_features:
    col1, col2 = feature, "rating"

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
    )
    # display(plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr"))
    display(
        plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")
    )


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

...

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)